<center><h1> Olivares Rueda Luis Carlos </h1></center> <br>
<center><h2> Double Pendulum </h2></center> <br>

<img src="Double_Pendulum.png" width="500" height="500">

Erase all data stored in the kernel.

In [52]:
%reset -f

In [53]:
# Numeric calculation libraries

import numpy as np
from scipy.integrate import odeint

# Symbolic calculation library

import sympy as smp

# Graph libraries

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

Define all the symbols needed to represent the problem.

In [54]:
# time and acceleration due to gravity constant (g)

t, g = smp.symbols('t g')

# Masses of the two particles of the pendulum

M1, M2 = smp.symbols('M1 M2')

# Lenghts of the two massless bars that joins the particles

l1, l2 = smp.symbols('l1 l2')

Define $\theta_{1}$ and $\theta_{2}$ as functions of time.

In [55]:
theta1 = smp.Function(r'\theta_1')(t)
theta2 = smp.Function(r'\theta_2')(t)

Define first and second derivatives of $\theta_{1}$ and $\theta_{2}$.

In [56]:
# First derivatives

theta1_d = smp.diff(theta1, t)
theta2_d = smp.diff(theta2, t)

# Second derivatives

theta1_dd = smp.diff(theta1_d, t)
theta2_dd = smp.diff(theta2_d, t)

Define the ($x$-$y$) position of the two particles of the pendulum.

In [57]:
# Position of particle 1 with mass M1

x1 = l1 * smp.sin(theta1)
y1 = -l1 * smp.cos(theta1)

# Position of particle 2 with mass M2

x2 = x1 + (l2 * smp.sin(theta2))
y2 = y1 - (l2 * smp.cos(theta2))

Define the velocities of the two particles.

In [58]:
# Velocity of particle 1

vx1 = smp.diff(x1, t)
vy1 = smp.diff(y1, t)

# Net velocity of particle 1

v1 = smp.sqrt((vx1**2) + (vy1**2))

# Velocity of particle 2

vx2 = smp.diff(x2, t)
vy2 = smp.diff(y2, t)

# Net velocity of particle 2

v2 = smp.sqrt((vx2**2) + (vy2**2))

Define the Langranian for each particle $L = K - V$ <br>
$K$ = Kinetic energy $= \frac{1}{2} m v^{2}$ <br>
$V$ = Potential energy $= m g y$ <br>
and the Langranian of the system is the sum of the Langranians of the particles.

In [59]:
# Langranian of particle 1

L1 = (smp.Rational(1, 2) * M1 * (v1**2)) - (M1 * g * y1)

# Langranian of particle 2

L2 = (smp.Rational(1, 2) * M2 * (v2**2)) - (M2 * g * y2)

# Langranian of the system

L = (L1 + L2).simplify()

Define the equations of motion by applying the Euler-Lagrange equations for $\theta_{1}$ and $\theta_{2}$ <br>
$\frac{d}{dt} \frac{\partial L}{\partial \dot{q}_{i}} - \frac{\partial L}{\partial q_{i}} = 0$

In [60]:
# Equation for theta1

Eq1 = smp.Eq(smp.diff(smp.diff(L, theta1_d), t) - smp.diff(L, theta1), 0).simplify()

# Equation for theta2

Eq2 = smp.Eq(smp.diff(smp.diff(L, theta2_d), t) - smp.diff(L, theta2), 0).simplify()

Solve the equations for $\frac{d^{2} \theta_{1}}{dt^{2}}$ and $\frac{d^{2} \theta_{2}}{dt^{2}}$

In [61]:
sols_dd = smp.solve([Eq1, Eq2], (theta1_dd, theta2_dd), simplify=True, rational=True)

Convert the system of two second order ODEs into a system of four first order ODEs <br>
<ul>
  <li>$\frac{d\theta_{1}}{dt} = z_{1}$</li>
  <li>$\frac{dz_{1}}{dt} = \frac{d^{2}\theta_{1}}{dt^{2}} = ...$</li>
  <li>$\frac{d\theta_{2}}{dt} = z_{2}$</li>
  <li>$\frac{dz_{2}}{dt} = \frac{d^{2}\theta_{2}}{dt^{2}} = ...$</li>
</ul> 
and transform the symbolic equations into numeric functions.

In [62]:
# z1

dtheta1_dt = smp.lambdify(theta1_d, theta1_d)

# dz1/dt

dz1_dt = smp.lambdify((t, g, M1, M2, l1, l2, theta1, theta2, theta1_d, theta2_d), sols_dd[theta1_dd])

# z2

dtheta2_dt = smp.lambdify(theta2_d, theta2_d)

# dz2/dt

dz2_dt = smp.lambdify((t, g, M1, M2, l1, l2, theta1, theta2, theta1_d, theta2_d), sols_dd[theta2_dd])

Define a vector $\vec{S} = (\theta_{1}, z_{1}, \theta_{2}, z_{2})$ and a function that takes $\vec{S}$ and $t$ and returns $\frac{d\vec{S}}{dt}$ in order to solve the system of ODEs in python.

In [63]:
def dS_dt(S, t, g, M1, M2, l1, l2):
    theta1, z1, theta2, z2 = S
    return [dtheta1_dt(z1), dz1_dt(t, g, M1, M2, l1, l2, theta1, theta2, z1, z2),
           dtheta2_dt(z2), dz2_dt(t, g, M1, M2, l1, l2, theta1, theta2, z1, z2)]

Define all the needed parameters and initial conditions and solve the system ODEs with the **odeint** method.

In [64]:
# Constant parameters

g = 9.8
M1 = 3
M2 = 5
l1 = 2.3
l2 = 0.7

# Initial conditions
# iv : initial velocity
# ip : initial position

# Initial conditions for theta1

ip_theta1 = np.pi
iv_theta1 = 0

# Initial conditions for theta2

ip_theta2 = 0.1 * np.pi
iv_theta2 = 0

# Initial vector

s0 = [ip_theta1, iv_theta1, ip_theta2, iv_theta2]

# it : initial time
# ft : final time
# steps: number of steps

it = 0
ft = 20
steps = 1001

t = np.linspace(it, ft, steps)

# Solve the system of equations

ans = odeint(dS_dt, y0=s0, t=t, args=(g, M1, M2, l1, l2))

Define a function that returns the position of the particles of the pendulum.

In [65]:
def x1_y1_x2_y2(t, theta1, theta2, l1, l2):
    return(l1 * np.sin(theta1), -l1 * np.cos(theta1), (l1 * np.sin(theta1)) + (l2 * np.sin(theta2)), (-l1 * np.cos(theta1)) - (l2 * np.cos(theta2)))

Calculate the positions.

In [66]:
X1, Y1, X2, Y2 = x1_y1_x2_y2(t, ans.T[0], ans.T[2], l1, l2)

Make an animation. <br>
Use your prefered library for animations.

In [67]:
my_frames = []
for i in range(len(X1) - 1):
        my_frames.append(go.Frame(data=[go.Scatter(x=[0, X1[i + 1]], y=[0, Y1[i + 1]]), go.Scatter(x=[X1[i + 1], X2[i + 1]], y=[Y1[i + 1], Y2[i + 1]])]))

In [68]:
fig = go.Figure(
    data=[go.Scatter(x=[0, X1[0]], y=[0, Y1[0]]), go.Scatter(x=[X1[0], X2[0]], y=[Y1[0], Y2[0]])],
    layout=go.Layout(
        xaxis=dict(range=[-l1 - l2 - 0.3, l1 + l2 + 0.3]),
        yaxis=dict(range=[-l1 - l2 - 0.3, l1 + l2 + 0.3]),
        title="Double Pendulum",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": ft, "redraw": False}, "fromcurrent": True, "transition": {"duration": 1 / len(t[t <= 1])
                                                                                                                   }}])])]
    ),
    frames=my_frames
)

fig.update_layout(width=800, height=800)

fig.show()